In [48]:
# Import Modules
import pandas as pd
import numpy as np
import plotly.express as px
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')


## 1. Data Exploration and Visualization

In [16]:
aggregated_data = pd.read_parquet("data/aggregated_data.parquet")
aggregated_data.head()

,OrderID,CustomerID,OrderDate,TotalAmount,FirstName,LastName,Email,Phone,CityName,State,OrderItemID,ProductID,Quantity,Price,ProductName,ProductDescription,ProductPrice,ProductInventory,CategoryName
0,1,29,2022-05-11 13:19:55.194747,16289.68,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,1,37,5,366.39,Ottoman,This is a Cabinet,366.39,81,Bedroom
1,1,29,2022-05-11 13:19:55.194747,16289.68,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,2,36,3,49.03,Recliner,This is a Armchair,49.03,51,Office
2,1,29,2022-05-11 13:19:55.194747,16289.68,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,4,41,5,418.29,Ottoman,This is a Ottoman,418.29,100,Office
3,1,29,2022-05-11 13:19:55.194747,16289.68,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,3,26,5,263.47,Chair,This is a Bed,263.47,3,Outdoor
4,1,29,2022-05-11 13:19:55.194747,16289.68,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,5,25,4,575.74,Ottoman,This is a Loveseat,575.74,2,Outdoor


In [17]:
aggregated_data.describe()

,OrderID,CustomerID,TotalAmount,OrderItemID,ProductID,Quantity,Price,ProductPrice,ProductInventory
count,2611.000000,2611.000000,2611.000000,2611.000000,2611.000000,2611.000000,2611.000000,2611.000000,2611.000000
mean,253.272309,51.037917,10495.333711,3.941785,25.655687,2.985063,507.491517,507.491517,53.332057
std,145.269093,29.104234,4886.808939,2.459258,14.396832,1.416436,259.499681,259.499681,26.312425
min,1.000000,1.000000,98.060000,1.000000,1.000000,1.000000,49.030000,49.030000,2.000000
25%,128.000000,25.000000,7112.100000,2.000000,13.000000,2.000000,286.620000,286.620000,38.000000
50%,252.000000,50.000000,10337.030000,3.000000,26.000000,3.000000,492.990000,492.990000,57.000000
75%,380.000000,77.000000,13851.420000,6.000000,38.000000,4.000000,675.200000,675.200000,71.000000
max,500.000000,100.000000,24587.050000,10.000000,50.000000,5.000000,994.550000,994.550000,100.000000


### 1.1 Top 10 Best-Selling Products

In [18]:
# Group order items by ProductName and sum the Quantity
product_sales = aggregated_data.groupby("ProductName")["Quantity"].sum().reset_index()

# Sort the products by Quantity sold in descending order
product_sales_sorted = product_sales.sort_values(by="Quantity", ascending=False)

# Group the order items by CategoryName and sum the quantity
category_sales = aggregated_data.groupby("CategoryName")["Quantity"].sum().reset_index()

# Display the top 10 best-selling products
top_10_products = product_sales_sorted.head(10)
print(top_10_products)

  ProductName  Quantity
3       Chair      1477
9       Table      1048
7    Recliner       953
6     Ottoman       871
8        Sofa       815
0    Armchair       757
5    Loveseat       747
1         Bed       497
4        Desk       370
2     Cabinet       259


In [19]:
fig = px.bar(top_10_products, x="ProductName", y="Quantity", title="Top 10 Best-Selling Products", labels={"ProductName": "Product Name", "Quantity": "Quantity Sold"})
fig.show()

### 1.2 Sales Distribution by Category

In [20]:
fig = px.pie(category_sales, values="Quantity", names="CategoryName", title="Sales Distribution by Category")
fig.show()

### 1.3 Total Sales Value Distribution per Customer

In [21]:
customer_sales = aggregated_data.groupby("CustomerID")["TotalAmount"].sum().reset_index()
fig = px.histogram(customer_sales, x="TotalAmount", nbins=20, labels={"TotalAmount": "Total Sales Value"})
fig.update_layout(title="Total Sales Value Distribution per Customer")
fig.show()

 ### 1.4 Number of Orders Distribution per Customer

In [22]:
orders_per_customer = aggregated_data.groupby("CustomerID")["OrderID"].nunique().reset_index()
fig = px.histogram(orders_per_customer, x="OrderID", nbins=20, labels={"OrderID": "Number of Orders"})
fig.update_layout(title="Number of Orders Distribution per Customer")
fig.show()

### 1.5 Sales by State

In [23]:
sales_by_state = aggregated_data.groupby("State")["Quantity"].sum().reset_index()
fig = px.bar(sales_by_state, x="State", y="Quantity", title="Sales by State", labels={"State": "State", "Quantity": "Quantity Sold"})
fig.show()

### 1.6 Correlation Matrix

In [24]:
numeric_columns = ["Quantity", "TotalAmount", "ProductPrice", "ProductInventory"]
correlation_matrix = aggregated_data[numeric_columns].corr()
fig = px.imshow(correlation_matrix, x=numeric_columns, y=numeric_columns, labels=dict(color="Correlação"))
fig.update_layout(title="Correlation Matrix")
fig.show()


### 1.7 Most Popular Product Categories by State

In [25]:
state_category = aggregated_data.groupby(["State", "CategoryName"])["Quantity"].sum().reset_index()
most_popular_categories = state_category.loc[state_category.groupby("State")["Quantity"].idxmax()]
fig = px.bar(most_popular_categories, x="State", y="Quantity", color="CategoryName", labels={"State": "State", "Quantity": "Quantity Sold", "CategoryName": "Category"})
fig.update_layout(title="Most Popular Product Categories by State")
fig.show()

## 2. Feature Engineering

### 2.1 Average amount spent per order for each customer

In [26]:
average_order_value = aggregated_data.groupby("CustomerID")["TotalAmount"].mean().reset_index()
average_order_value.columns = ["CustomerID", "AverageOrderValue"]
average_order_value

,CustomerID,AverageOrderValue
0,1,8666.425000
1,2,15221.470000
2,3,7889.331538
3,4,6880.284000
4,5,11482.436341
...,...,...
95,96,11439.750000
96,97,9229.196429
97,98,6387.680000
98,99,7378.192500


### 2.2 Average number of products purchased per order for each customer

In [27]:
average_quantity = aggregated_data.groupby("CustomerID")["Quantity"].mean().reset_index()
average_quantity.columns = ["CustomerID", "AverageQuantity"]
average_quantity

,CustomerID,AverageQuantity
0,1,3.000000
1,2,2.777778
2,3,2.692308
3,4,2.466667
4,5,3.024390
...,...,...
95,96,3.107143
96,97,2.500000
97,98,3.400000
98,99,2.125000


### 2.3 Total sales per customer:

In [28]:
total_sales = aggregated_data.groupby("CustomerID")["TotalAmount"].sum().reset_index()
total_sales.columns = ["CustomerID", "TotalSales"]
total_sales

,CustomerID,TotalSales
0,1,155995.65
1,2,136993.23
2,3,102561.31
3,4,103204.26
4,5,470779.89
...,...,...
95,96,320313.00
96,97,129208.75
97,98,95815.20
98,99,59025.54


### 2.4 Total orders per customer:

In [29]:
total_orders = aggregated_data.groupby("CustomerID")["OrderID"].nunique().reset_index()
total_orders.columns = ["CustomerID", "TotalOrders"]
total_orders


,CustomerID,TotalOrders
0,1,4
1,2,1
2,3,2
3,4,4
4,5,7
...,...,...
95,96,4
96,97,3
97,98,4
98,99,2


### 2.5 Aggregate the new features

In [30]:
customer_features = aggregated_data.merge(average_order_value, on="CustomerID").merge(average_quantity, on="CustomerID").merge(total_sales, on="CustomerID").merge(total_orders, on="CustomerID")
customer_features

,OrderID,CustomerID,OrderDate,TotalAmount,FirstName,LastName,Email,Phone,CityName,State,OrderItemID,ProductID,Quantity,Price,ProductName,ProductDescription,ProductPrice,ProductInventory,CategoryName,AverageOrderValue,AverageQuantity,TotalSales,TotalOrders
0,1,29,2022-05-11 13:19:55.194747,16289.68,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,1,37,5,366.39,Ottoman,This is a Cabinet,366.39,81,Bedroom,13348.618125,3.343750,427155.78,5
1,1,29,2022-05-11 13:19:55.194747,16289.68,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,2,36,3,49.03,Recliner,This is a Armchair,49.03,51,Office,13348.618125,3.343750,427155.78,5
2,1,29,2022-05-11 13:19:55.194747,16289.68,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,4,41,5,418.29,Ottoman,This is a Ottoman,418.29,100,Office,13348.618125,3.343750,427155.78,5
3,1,29,2022-05-11 13:19:55.194747,16289.68,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,3,26,5,263.47,Chair,This is a Bed,263.47,3,Outdoor,13348.618125,3.343750,427155.78,5
4,1,29,2022-05-11 13:19:55.194747,16289.68,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,5,25,4,575.74,Ottoman,This is a Loveseat,575.74,2,Outdoor,13348.618125,3.343750,427155.78,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2606,476,86,2022-12-29 13:19:55.198906,3030.04,Robert,Smith,customer86@example.com,123-456-0086,Salvador,SP,1,31,4,147.79,Recliner,This is a Table,147.79,52,Office,8351.445769,2.615385,217137.59,6
2607,476,86,2022-12-29 13:19:55.198906,3030.04,Robert,Smith,customer86@example.com,123-456-0086,Salvador,SP,2,13,3,668.39,Chair,This is a Table,668.39,24,Outdoor,8351.445769,2.615385,217137.59,6
2608,479,86,2022-04-18 13:19:55.198931,1727.22,Robert,Smith,customer86@example.com,123-456-0086,Salvador,SP,1,25,3,575.74,Ottoman,This is a Loveseat,575.74,2,Outdoor,8351.445769,2.615385,217137.59,6
2609,497,86,2022-10-12 13:19:55.199093,2463.57,Robert,Smith,customer86@example.com,123-456-0086,Salvador,SP,2,41,4,418.29,Ottoman,This is a Ottoman,418.29,100,Office,8351.445769,2.615385,217137.59,6


## 3. Data Pre-Processing

### 3.1 Check and handle missing values

In [31]:
missing_values = customer_features.isnull().sum()
missing_values


OrderID               0
CustomerID            0
OrderDate             0
TotalAmount           0
FirstName             0
LastName              0
Email                 0
Phone                 0
CityName              0
State                 0
OrderItemID           0
ProductID             0
Quantity              0
Price                 0
ProductName           0
ProductDescription    0
ProductPrice          0
ProductInventory      0
CategoryName          0
AverageOrderValue     0
AverageQuantity       0
TotalSales            0
TotalOrders           0
dtype: int64

### 3.2 Handle outliers:

In [32]:
Q1 = customer_features["TotalAmount"].quantile(0.25)
Q3 = customer_features["TotalAmount"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

customer_features = customer_features[(customer_features["TotalAmount"] >= lower_bound) & (customer_features["TotalAmount"] <= upper_bound)]
customer_features


,OrderID,CustomerID,OrderDate,TotalAmount,FirstName,LastName,Email,Phone,CityName,State,OrderItemID,ProductID,Quantity,Price,ProductName,ProductDescription,ProductPrice,ProductInventory,CategoryName,AverageOrderValue,AverageQuantity,TotalSales,TotalOrders
0,1,29,2022-05-11 13:19:55.194747,16289.68,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,1,37,5,366.39,Ottoman,This is a Cabinet,366.39,81,Bedroom,13348.618125,3.343750,427155.78,5
1,1,29,2022-05-11 13:19:55.194747,16289.68,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,2,36,3,49.03,Recliner,This is a Armchair,49.03,51,Office,13348.618125,3.343750,427155.78,5
2,1,29,2022-05-11 13:19:55.194747,16289.68,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,4,41,5,418.29,Ottoman,This is a Ottoman,418.29,100,Office,13348.618125,3.343750,427155.78,5
3,1,29,2022-05-11 13:19:55.194747,16289.68,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,3,26,5,263.47,Chair,This is a Bed,263.47,3,Outdoor,13348.618125,3.343750,427155.78,5
4,1,29,2022-05-11 13:19:55.194747,16289.68,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,5,25,4,575.74,Ottoman,This is a Loveseat,575.74,2,Outdoor,13348.618125,3.343750,427155.78,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2606,476,86,2022-12-29 13:19:55.198906,3030.04,Robert,Smith,customer86@example.com,123-456-0086,Salvador,SP,1,31,4,147.79,Recliner,This is a Table,147.79,52,Office,8351.445769,2.615385,217137.59,6
2607,476,86,2022-12-29 13:19:55.198906,3030.04,Robert,Smith,customer86@example.com,123-456-0086,Salvador,SP,2,13,3,668.39,Chair,This is a Table,668.39,24,Outdoor,8351.445769,2.615385,217137.59,6
2608,479,86,2022-04-18 13:19:55.198931,1727.22,Robert,Smith,customer86@example.com,123-456-0086,Salvador,SP,1,25,3,575.74,Ottoman,This is a Loveseat,575.74,2,Outdoor,8351.445769,2.615385,217137.59,6
2609,497,86,2022-10-12 13:19:55.199093,2463.57,Robert,Smith,customer86@example.com,123-456-0086,Salvador,SP,2,41,4,418.29,Ottoman,This is a Ottoman,418.29,100,Office,8351.445769,2.615385,217137.59,6


### 3.3 Scaling and normalization

In [33]:
numeric_columns = ["TotalAmount", "TotalOrders", "AverageOrderValue", "AverageQuantity"]
scaler = StandardScaler()
customer_features[numeric_columns] = scaler.fit_transform(customer_features[numeric_columns])
customer_features

,OrderID,CustomerID,OrderDate,TotalAmount,FirstName,LastName,Email,Phone,CityName,State,OrderItemID,ProductID,Quantity,Price,ProductName,ProductDescription,ProductPrice,ProductInventory,CategoryName,AverageOrderValue,AverageQuantity,TotalSales,TotalOrders
0,1,29,2022-05-11 13:19:55.194747,1.237941,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,1,37,5,366.39,Ottoman,This is a Cabinet,366.39,81,Bedroom,1.259609,1.245064,427155.78,-0.410164
1,1,29,2022-05-11 13:19:55.194747,1.237941,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,2,36,3,49.03,Recliner,This is a Armchair,49.03,51,Office,1.259609,1.245064,427155.78,-0.410164
2,1,29,2022-05-11 13:19:55.194747,1.237941,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,4,41,5,418.29,Ottoman,This is a Ottoman,418.29,100,Office,1.259609,1.245064,427155.78,-0.410164
3,1,29,2022-05-11 13:19:55.194747,1.237941,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,3,26,5,263.47,Chair,This is a Bed,263.47,3,Outdoor,1.259609,1.245064,427155.78,-0.410164
4,1,29,2022-05-11 13:19:55.194747,1.237941,John,Rodriguez,customer29@example.com,123-456-0029,Recife,PR,5,25,4,575.74,Ottoman,This is a Loveseat,575.74,2,Outdoor,1.259609,1.245064,427155.78,-0.410164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2606,476,86,2022-12-29 13:19:55.198906,-1.547965,Robert,Smith,customer86@example.com,123-456-0086,Salvador,SP,1,31,4,147.79,Recliner,This is a Table,147.79,52,Office,-0.934913,-1.276463,217137.59,0.071519
2607,476,86,2022-12-29 13:19:55.198906,-1.547965,Robert,Smith,customer86@example.com,123-456-0086,Salvador,SP,2,13,3,668.39,Chair,This is a Table,668.39,24,Outdoor,-0.934913,-1.276463,217137.59,0.071519
2608,479,86,2022-04-18 13:19:55.198931,-1.821693,Robert,Smith,customer86@example.com,123-456-0086,Salvador,SP,1,25,3,575.74,Ottoman,This is a Loveseat,575.74,2,Outdoor,-0.934913,-1.276463,217137.59,0.071519
2609,497,86,2022-10-12 13:19:55.199093,-1.666983,Robert,Smith,customer86@example.com,123-456-0086,Salvador,SP,2,41,4,418.29,Ottoman,This is a Ottoman,418.29,100,Office,-0.934913,-1.276463,217137.59,0.071519


## 4. Model Selection

### 4.1 Testing Clustering Algorithms 

In [34]:
# Selecione as colunas relevantes
selected_columns = ["TotalOrders", "TotalSales", "AverageOrderValue", "AverageQuantity"]
X = customer_features[selected_columns]

# K-Means
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans_labels = kmeans.fit_predict(X)

# DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(X)

# Agglomerative Clustering
agg_clustering = AgglomerativeClustering(n_clusters=3)
agg_labels = agg_clustering.fit_predict(X)

# HDBSCAN
# hdbscan_cluster = hdbscan.HDBSCAN(min_cluster_size=5)
# hdbscan_labels = hdbscan_cluster.fit_predict(X)

# Avalie os resultados usando Silhouette Score e Calinski-Harabasz Index
algorithms = ["K-Means", "DBSCAN", "Agglomerative Clustering"]  # Adicione "HDBSCAN" se estiver usando-o
labels_list = [kmeans_labels, dbscan_labels, agg_labels]  # Adicione hdbscan_labels se estiver usando-o

for algorithm, labels in zip(algorithms, labels_list):
    if len(np.unique(labels)) > 1:  # Verifique se há mais de um cluster
        silhouette = silhouette_score(X, labels)
        calinski_harabasz = calinski_harabasz_score(X, labels)
    else:
        silhouette = calinski_harabasz = "N/A"
    
    print(f"{algorithm}:")
    print(f"Silhouette Score: {silhouette}")
    print(f"Calinski-Harabasz Index: {calinski_harabasz}\n")


K-Means:
Silhouette Score: 0.6038884850367526
Calinski-Harabasz Index: 9190.27119818303

DBSCAN:
Silhouette Score: 1.0
Calinski-Harabasz Index: 1.3149861881887704e+31

Agglomerative Clustering:
Silhouette Score: 0.554231286089753
Calinski-Harabasz Index: 7264.985686283593



### 4.2 Plot for Each Clustering Algorithm

In [35]:
def plot_clusters_3d(data, labels, algorithm):
    data["Cluster"] = labels
    fig = px.scatter_3d(data, x='TotalOrders', y='TotalSales', z='AverageOrderValue',
                        color='Cluster', text='CustomerID', title=f'{algorithm} Clustering',
                        labels={'TotalOrders': 'Total Orders', 'TotalSales': 'Total Sales',
                                'AverageOrderValue': 'Average Order Value'},
                        hover_data=['FirstName', 'LastName', 'Email', 'Phone', 'CityName', 'State'])
    fig.show()

plot_clusters_3d(customer_features, kmeans_labels, 'K-Means')
plot_clusters_3d(customer_features, dbscan_labels, 'DBSCAN')
plot_clusters_3d(customer_features, agg_labels, 'Agglomerative Clustering')


## 5. Model Tuning

### 5.1 Hyperparameter optimizing

In [36]:
# K-Means hyperparameter tuning
kmeans_results = []

for n_clusters in range(2, 11):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans_labels = kmeans.fit_predict(X)
    
    silhouette = silhouette_score(X, kmeans_labels)
    calinski_harabasz = calinski_harabasz_score(X, kmeans_labels)
    
    kmeans_results.append({
        'n_clusters': n_clusters,
        'silhouette': silhouette,
        'calinski_harabasz': calinski_harabasz
    })

kmeans_results_df = pd.DataFrame(kmeans_results)
print("K-Means results:")
print(kmeans_results_df)


K-Means results:
   n_clusters  silhouette  calinski_harabasz
0           2    0.599475        5963.290608
1           3    0.603888        9190.271198
2           4    0.609818       10382.007119
3           5    0.632094       15335.919435
4           6    0.635630       19999.911799
5           7    0.670983       24066.929447
6           8    0.684074       33445.100634
7           9    0.685408       35312.391450
8          10    0.668700       38795.815646


In [37]:
# DBSCAN hyperparameter tuning
dbscan_results = []

eps_values = np.arange(0.1, 1.0, 0.1)
min_samples_values = range(2, 11)

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        dbscan_labels = dbscan.fit_predict(X)
        
        n_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
        
        if n_clusters > 1:
            silhouette = silhouette_score(X, dbscan_labels)
            calinski_harabasz = calinski_harabasz_score(X, dbscan_labels)
        else:
            silhouette = calinski_harabasz = float('nan')
        
        dbscan_results.append({
            'eps': eps,
            'min_samples': min_samples,
            'n_clusters': n_clusters,
            'silhouette': silhouette,
            'calinski_harabasz': calinski_harabasz
        })

dbscan_results_df = pd.DataFrame(dbscan_results)
print("\nDBSCAN results:")
print(dbscan_results_df)



DBSCAN results:
    eps  min_samples  n_clusters  silhouette  calinski_harabasz
0   0.1            2         100    1.000000       1.314986e+31
1   0.1            3         100    1.000000       1.314986e+31
2   0.1            4         100    1.000000       1.314986e+31
3   0.1            5         100    1.000000       1.314986e+31
4   0.1            6         100    1.000000       1.314986e+31
..  ...          ...         ...         ...                ...
76  0.9            6         100    1.000000       1.314986e+31
77  0.9            7          98    0.996966       9.439950e+05
78  0.9            8          97    0.992507       3.565602e+05
79  0.9            9          96    0.991525       2.332971e+05
80  0.9           10          93    0.973913       2.622097e+04

[81 rows x 5 columns]


In [38]:
# Agglomerative Clustering hyperparameter tuning
agg_results = []

for n_clusters in range(2, 11):
    agg_clustering = AgglomerativeClustering(n_clusters=n_clusters)
    agg_labels = agg_clustering.fit_predict(X)
    
    silhouette = silhouette_score(X, agg_labels)
    calinski_harabasz = calinski_harabasz_score(X, agg_labels)
    
    agg_results.append({
        'n_clusters': n_clusters,
        'silhouette': silhouette,
        'calinski_harabasz': calinski_harabasz
    })

agg_results_df = pd.DataFrame(agg_results)
print("\nAgglomerative Clustering results:")
print(agg_results_df)


Agglomerative Clustering results:
   n_clusters  silhouette  calinski_harabasz
0           2    0.576079        4613.896113
1           3    0.554231        7264.985686
2           4    0.582535       10278.662567
3           5    0.632450       14718.901738
4           6    0.624285       17614.685088
5           7    0.675320       21730.126837
6           8    0.685665       33316.717461
7           9    0.686576       34778.431687
8          10    0.675160       37927.289581


In [39]:
# Create a new DataFrame with customer ID and cluster labels
customer_clusters = pd.DataFrame({"CustomerID": customer_features.index, "Cluster": dbscan_labels})
customer_clusters.head()


,CustomerID,Cluster
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [40]:
# Merge the customer clusters with the original dataset
merged_data = aggregated_data.merge(customer_clusters, on="CustomerID")

# Analyze the characteristics of each cluster
cluster_summary = merged_data.groupby("Cluster").agg(
    {
        "TotalAmount": ["mean", "sum"],
        "Quantity": ["mean", "sum"],
        "ProductID": "nunique",
        "CustomerID": "nunique",
    }
)
cluster_summary.columns = ["_".join(col).strip() for col in cluster_summary.columns.values]
cluster_summary.reset_index(inplace=True)
print(cluster_summary)


   Cluster  TotalAmount_mean  TotalAmount_sum  Quantity_mean  Quantity_sum  \
0        0      10162.724892       8455387.11       2.955529          2459   
1        1      10745.185234       7349706.70       3.013158          2061   
2        2      11029.586845       7831006.66       3.022535          2146   
3        3       9784.976234       3767215.85       2.929870          1128   

   ProductID_nunique  CustomerID_nunique  
0                 50                  31  
1                 50                  28  
2                 50                  28  
3                 50                  13  


## 6. Conclusion

Based on the analysis conducted throughout this project, we can draw the following conclusions:

**Data Exploration and Visualization:** We have successfully explored and visualized the transaction data, gaining insights into the customer purchase behavior, product categories, and regional distribution of customers. This helped us to identify potential customer segments and choose relevant features for clustering.

**Feature Engineering:** We created new features such as TotalOrders, TotalSales, AverageOrderValue, and AverageQuantity, which are essential for understanding customer behavior and preferences.

**Data Preprocessing:** We handled missing values, outliers, and applied necessary data transformations such as encoding categorical variables and scaling numerical features. This prepared our dataset for clustering analysis.

**Model Selection and Evaluation:** We tested multiple clustering algorithms (K-Means, DBSCAN, and Agglomerative Clustering) and evaluated their performance using Silhouette Score and Calinski-Harabasz Index. Based on the evaluation, we selected DBSCAN as the best clustering algorithm for our dataset.

**Model Tuning**: We optimized the hyperparameters of the DBSCAN algorithm to improve its clustering performance further.

**Final Clustering and Analysis**: The optimized DBSCAN model identified four distinct customer segments (Cluster 0, Cluster 1, Cluster 2, and Cluster 3). We analyzed the characteristics of each cluster, such as the average total amount spent, average quantity purchased, and the number of unique products and customers.

With these insights, Natuzzi can develop personalized marketing campaigns and tailor product offerings to better serve each customer segment, ultimately increasing customer satisfaction, loyalty, and sales. The company can also continuously update and refine the clustering model as new transaction data becomes available, ensuring that the customer segmentation remains relevant and accurate over time.